In [ ]:
# """
# 把資料mapping至google drive
# 並且把資料寫在/content/drive
# 如此一來資料即可保存在google drive
# 執行後會需要驗證身份，並輸入驗證碼

# ***** 只有在開發階段才會用這種保存到google drive的方式，不然基本上都會將資料寫進資料庫中 *****

# """

# from google.colab import drive

# # 設置路徑
# drive.mount("/content/drive")


In [ ]:
"""
安裝套件(line-bot-sdk, flask, flask-ngrok)
"""
!pip install line-bot-sdk flask flask-ngrok

     |████████████████████████████████| 71kB 2.0MB/s 


In [ ]:
"""
引用套件
"""

# 引用Flask Web Server套件
from flask import Flask, request, abort, jsonify

# 引用json套件
import json

# 引用生成外部連結的flask-ngrok套件，裡面的run_with_ngrok功能
from flask_ngrok import run_with_ngrok

# 引用linebot套件包裡的LineBotApi跟WebhookHandler
from linebot import LineBotApi, WebhookHandler

# 引用linebot套件包的exceptions功能裡面的InvalidSignatureError (無效簽章錯誤)
from linebot.exceptions import InvalidSignatureError

In [ ]:
"""
建置主程序 (這裡指的就是line機器人)

分別建置:
1. handler: 公司用來接收用戶傳遞的一切訊息，並順便做驗證過濾的單位
  (如同民眾郵購時，商店會有前台小秘書來過濾分派信件，這裡的小秘書=handler)


2. line_bot_api: 公司要傳訊給民眾時，公司會委託業務將信件寄到郵局，郵局再寄到用戶手上 
  (如同貨物公司收到民眾郵購憑單時，會請業務將收據送到郵局，郵局再將收據寄給用戶，這裡的業務=line_bot_api ；郵局=LINE)
  (取個資也是這個階段)


建置方式:
1. 先準備好APP(flask web server)
2. 建置Handler
3. 準備Line_bot_api

"""

# 先準備APP，並啟用 (即設定Flask Web Server啟用細節)
app = Flask(__name__, static_url_path="/material", static_folder="./material/")
run_with_ngrok(app)


# 生成實體物件(handler跟line_bot_api)
"""
藉由linebot裡面的
- WebhookHandler("Channel Secret")
- LineBotApi("Channel access token")

** 兩個密鑰請到LINE Developer Console去尋找 **

"""

line_bot_api = LineBotApi("HPNN2U0LfdttfMl7b5yCkW/h9yzuxYOGW4NYDnJuBD4qXTY/UagvGJK3OKAnGwQwgA1tYG9lPNFt92N+j33osr3nQ59os0a/bD+usRNyUumvLs0IYnHTR6FkmhCAaQ8WkcKZq6a3AJlA4tfw9jxLtgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("e20a1f0d6418dd3bebb261a9fb9374ab")

In [ ]:
"""
重點一: 處理訊息


建置主程序的API接口 (消息的處理流程如下)

1. 接收用戶傳過來的訊息 (webhookhandler)

2. 將訊息取出並存在google drive的檔案內 (開發時才會這樣使用，不然一般會跟資料庫做連動)

3. 接收完用戶訊息後要進行驗證分派，然後在將回覆回傳給用戶 (line_bot_api → LINE)


當用戶發訊息給ngrok.xxxxx/時，就會觸發LINE消息的處理流程

"""

# 啟動server的對外接口，讓民眾能丟訊息進來(最簡單)
@app.route("/", methods=["POST"])
def callback():
  
  # 1. 取得傳遞訊息的LINE加密簽章(我要檢查你的headers啦，跟爬蟲概念類似)    =>  檢查信件戳章
  signature = request.headers["X-Line-Signature"]

  # 2. 取得訊息的內容為何(最好印出來看看)                    =>  查看信件內容
  body = request.get_data(as_text=True)
  print(body) 

  # 3. 紀錄用戶的log(痕跡) (存放在google drive是不得已，一般會存放在資料庫)  =>  前台留下收件紀錄
  f = open("/content／drive/MyDrive/ai-event.log", "a")
  f.write(body)
  f.close()


  # 4. handler依據訊息內容分派訊息的去向                    =>  將信件轉交給負責信中提及業務的人
  try:
      """
      給handler做檢查，handler會依據不同的Event做處理
      webhookhandler物件.handle(訊息內容, 訊息的戳章)
      """
      handler.handle(body, signature) 
  
  except InvalidSignatureError:
      abort(400)

  return 'OK'

In [ ]:

"""
重點二: 處理關注事件


*** 用戶關注事件 ==> FollowEvent

用戶關注時，商業邏輯？

1. 先取得用戶個資，並存回伺服器(此為存放到google drive)
2. 再回應用戶，發表感謝心得文

"""

################################################################################
"""
事前作業:
"""
# 先引用linebot.models.events裡面的FollowEvent功能  (關注事件)
from linebot.models.events import FollowEvent

# 接著引用linebot.models裡面的TextMessage跟TextSendMessag  (用戶傳來跟回傳用戶)
from linebot.models import TextMessage, TextSendMessage

################################################################################

"""
正式作業: 
"""
# 告知handler，如果收到FollowEvent，則做下面的方法處理  (告知前台，如果收到的信件內容業務是關注事件，就按照下面的流程辦事)

@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
  
  # 先取出用戶個資
  user_profile = line_bot_api.get_profile(event.source.user_id)
  """
  - event:  每次傳來的訊息
  - source： 訊息來源
  - user_id: 發訊息的用戶ID
  => event.source.user_id (記憶法：訊息來源的用戶id)
  """

  # 然後存放個資(此為存放到google drive)
  with open("/content/drive/MyDrive/users.txt", "a") as myfile:
    myfile.write(json.dumps(vars(user_profile), sort_keys=True))
    myfile.write("\n")

  
  # 提取完個資後，line_bot_api回應消息給LINE，LINE再回應消息或圖片給用戶(非必要)
  line_bot_api.reply_messaage(event.reply_token, [TextSendMessage("安安，你的個資已被我記錄了喔!")])
  
  # 歡迎訊息的提供：
  # line_bot_api.reply_message(event.reply_token, TextSendMessage("您好" + user_profile.display_name))

  """
  reply_message(event.reply_token, [TextSendMessage(回應給用戶的文字)])
  - reply_token  =>  每一次用戶傳訊息給LINE，LINE轉傳過來時，都會附上一個票券(確認身份)，我們得拿這個票券跟LINE回應
  - TextSendMessage(text="回應給用戶的文字")

  => event.reply_token (記憶法：訊息的寄件人地址)
  """


In [ ]:
# 運行主程序
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://91fbe1c6a7a5.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
